In [2]:
# import libraries
import pandas as pd
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# disable warnings, use w caution
import warnings
warnings.filterwarnings('ignore')

# project specific libs
import os
import matplotlib.pyplot as plt

In [3]:
# set (local) path
path = '/Users/KevinBu/Desktop/clemente_lab/Projects/ampaim/'

In [4]:
###
# Load AB mapping file
###

# get AMPAIM+EISER mapping file w metadata; drop row 1
df_map_AB = pd.read_csv(path + 'inputs/adamcantor22_Cross_Disease_Pilot_0/Qiime2_0/qiime_mapping_file.tsv', sep='\t', header=0, index_col=0)

# get first row for any future operations
first_row = df_map_AB.reset_index().iloc[0,]
first_row_df = pd.DataFrame(first_row).T
df_map_AB = df_map_AB.iloc[1:,]


###
# Specific replacements
###

# 526-0-twin-psaplate308 is missing a dash
df_map_AB = df_map_AB.rename(index={'526-0-twin-psaplate308': '526-0-twin-psa-plate308'})

###
# General replacements
###

# refrain from dropping NA's because you might need the columns for Q2 and you need to smoothly concat the first row

# convert index to str from float etc.
df_map_AB.index = df_map_AB.index.map(str)

# switch EISER diagnosis to eiser from NA
df_map_AB.loc[df_map_AB['Project'] == 'eiser', 'Diagnosis'] = 'eiser'

# RBB mapping file doesn't have info on duplicate controls, and the glass control info is extracted only from a subset of the names
# so we need to grab that info from AB's mapping file
samp_to_ctrlstatus = df_map_AB['Project'].to_dict()

# grab AB non-microteach samples
df_map_AB_filt = df_map_AB[df_map_AB['Project'].isin(['eiser','TWIN_PSA','glass_control','neg_control','duplicate_control'])]

df_map_AB_filt.head()

,BarcodeSequence,LinkerPrimerSequence,AmpliconWell,BSA,BSASeverityByBSA,CCPtiter,CRP,CurrentBiologics,CurrentIntralesionalSteroids,CurrentMTX,...,StudyType,SubjectType,BirthYear,HostSubjectId,Nationality,Sex,SpecimenType,UberonCodeType,Weight,WeightDateCollected
#SampleID,,,,,,,,,,,,,,,,,,,,,
116783-plate305,CCTCGCATGACC,CCGGACTACHVGGGTWTCTAAT,A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Human,NaN,116783,NaN,NaN,fecal,NaN,NaN,NaN
116784-plate305,CGCGCAAGTATT,CCGGACTACHVGGGTWTCTAAT,B1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Human,NaN,116784,NaN,NaN,fecal,NaN,NaN,NaN
116785-plate305,AAGGCGCTCCTT,CCGGACTACHVGGGTWTCTAAT,C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Human,NaN,116785,NaN,NaN,fecal,NaN,NaN,NaN
116786-plate305,CGCAATGAGGGA,CCGGACTACHVGGGTWTCTAAT,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Human,NaN,116786,NaN,NaN,fecal,NaN,NaN,NaN
116787-plate305,ACGGCGTTATGT,CCGGACTACHVGGGTWTCTAAT,E1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Human,NaN,116787,NaN,NaN,fecal,NaN,NaN,NaN


In [5]:
###
# Prepare for merge with RB mapping file
###

# load RB mapping file
df_map_RB = pd.read_csv(path + 'inputs/qiime_mapping_file_final_062123_RBB_ESR_CRPmod.csv', index_col=0)

# skip row with q2types; 140 columns
df_map_RB = df_map_RB.iloc[1:,]

# exclude unaffected since we are using TWINS_PSA from AB and controls from AB
# we only want microteach samples
df_map_RB = df_map_RB[df_map_RB['Diagnosis'].isin(['healthy','ss','sle','cd','RA','psa','pso'])]

# drop duplicate samples that appear in AB and that are controls per AB mapping file
for s in list(df_map_RB.index.values):
    if samp_to_ctrlstatus[s] in ['glass_control','duplicate_control','neg_control']:
        df_map_RB = df_map_RB.drop(s)
        
        
# merge with AB
df_meta = pd.concat([df_map_AB_filt, df_map_RB], axis=0)

# drop 'Separate' and 'Together'; presumably Q2 artifacts
df_meta = df_meta.drop(['Separate','Together'], axis=1)

# add in Project col from AB that describes controls
df_meta['Project'] = df_meta.index.map(samp_to_ctrlstatus)

###
# Specific replacements
###

# replace HxOtherDMARDs
df_meta.loc['540-0-twin-psa-plate308','HxOtherDMARDs'] = 'N'

### 
# General replacements
###

# specify float cols for later
float_cols = ['Age','BSA','CRP','DAS28','ESR','PhysicianGlobalPsA','RAPID3','SJC','TJC']

# Replace Y,N with 1,0 
df_meta = df_meta.replace({'Y':1,'N':0})

# create dict mapping diagnosis to samples
diag_to_samp = {}
for d in list(set(df_meta['Diagnosis'].values)):
    diag_to_samp[d] = list(df_meta[df_meta['Diagnosis'] == d].index.values)

### 
# Create new mapping files
### 

def export_q2(df, first_row_df):
    df_q2 = df.reset_index()
    df_q2 = pd.concat([first_row_df, df_q2])
    df_q2 = df_q2.set_index('#SampleID')
    df_q2.loc['#q2:types',:] = 'categorical'
    df_q2 = df_q2.reset_index()
    return df_q2

# (1) filter out controls
df_meta = df_meta[df_meta['Project'].isin(['eiser','microteach','TWIN_PSA'])]

# (2) filter out treatments
# drop TWIN PSA and PSO samples that have treatment on biologics, DMARDS, MTX
drop_samples = ['275-psa-plate307', '475-psa-plate307', '542-0-twin-psa-plate308']
df_meta = df_meta.drop(drop_samples)

# remove samples that are treated other than above
drops = []
for v in ['CurrentBiologics', 'CurrentOtherDMARDs', 'CurrentMTX', 'HxOtherDMARDs']:
    df = df_meta[df_meta[v].isin(['Y',1])]
    drops = drops + list(df.index.values)

df_meta = df_meta.drop(drops)
print('Dropped this many samples: ' + str(len(drops)))

# drop eiser samples from downstream analysis
df_meta = df_meta[df_meta['Diagnosis'] != 'eiser']

# keep only psa
df_meta = df_meta[df_meta['Diagnosis'] == 'psa']

# export to Q2
df_meta_q2 = export_q2(df_meta, first_row_df)
df_meta_q2.to_csv(path + 'inputs/qiime_mapping_file_noctrl_noeiser_psaonly.tsv', sep='\t', index=False, na_rep='nan') 

###
# Pre-processing
###

df_meta['Diagnosis'].value_counts()

Dropped this many samples: 18


Diagnosis
psa    27
Name: count, dtype: int64

In [6]:
# merge CDP and MSQ141 MMEDs qiime mapping files
# load CDP
df_cdp = pd.read_csv(path + 'inputs/qiime_mapping_file_noctrl_noeiser_psaonly.tsv', sep='\t')
df_cdp['Batch'] = 'CDP'

# load MSQ141
df_141 = pd.read_csv(path + 'inputs/qiime_mapping_file_MSQ141.tsv', sep='\t')

# drop row 0 (#q2types) prior to merge
df_141 = df_141.iloc[1:,:]
df_141['Batch'] = 'MSQ141'

# add in diagnosis column that is IllnessNotes to all lowercase
df_141['Diagnosis'] = df_141['IllnessNotes'].apply(lambda x: x.lower())

# replace sjd with ss and psa_pso with psa
df_141['Diagnosis'] = df_141['Diagnosis'].replace('sjd','ss')
df_141['Diagnosis'] = df_141['Diagnosis'].replace('psa_pso','psa')
df_141['Diagnosis'] = df_141['Diagnosis'].replace('ra','RA')

# merge rows
df_merge = pd.concat([df_cdp, df_141])#, axis=1)
df_merge = df_merge.set_index('#SampleID')
df_merge.loc['#q2:types',:] = 'categorical'

# note that lots of reasons might exist for batch effect, like treatment, also different diseases
# columns are not consistent (e.g. IllnessNotes in MSQ141 vs Diagnosis in CDP)
df_merge.to_csv(path + 'inputs/qiime_mapping_file_batch_nocd.tsv', sep='\t')

# standard binarize column for each medication
# convert from float
df_merge['Medication'] = df_merge['Medication'].astype(str)

# grab all medications
meds = list(df_merge['Medication'].values)

# drop 'categorical'
meds.remove('categorical')

uniq_meds = []
# split items with '_'
for m in meds:
    if '_' in m:
        items = m.split('_')
        items = [i.lower() for i in items]
        uniq_meds = uniq_meds + items
    else:
        uniq_meds.append(m.lower())
    
uniq_meds = list(set(list(uniq_meds)))
print(uniq_meds)

# create new column for each med
for u in uniq_meds:
    df_merge[u] = 0
    df_merge.loc[0,u] = 'categorical'

# iterate through each sample
for index, row in df_merge.iterrows():
    if index != 0: # the whole '#q2:types', 'categorical' thing
        medications = df_merge.loc[index,'Medication'].lower()
        medications = medications.split('_')
        for m in medications:    
            df_merge.loc[index,m] = 1

# new mapping file with med columns
df_merge = df_merge[df_merge['Diagnosis'] == 'psa']

# keep only those with psa data
df_merge = df_merge.dropna(subset='PsAtype')# [df_merge['PsAtype']


# create a new column 'Involvement' that is 'Axial' if PsAtype is 'peripheral_axial', 'axial', or 'Spine involvement'
# print(df_merge['PsAtype'].value_counts())
df_merge['Involvement'] = df_merge['PsAtype'].apply(lambda x: 'peripheral' if x == 'peripheral' else 'axial')
# print(df_merge['Involvement'].value_counts()) 19 periph to 8 axial

df_merge_q2 = export_q2(df_merge, first_row_df)
df_merge_q2.to_csv(path + 'inputs/qiime_mapping_file_merge_meds_psaonly.tsv', sep='\t', index=False, na_rep='nan') 

df_merge.head()

['prednisone', 'leflunomide', 'nomed', 'mtx', 'nan', 'topicals', 'hcq']


,BarcodeSequence,LinkerPrimerSequence,AmpliconWell,BSA,BSASeverityByBSA,CCPtiter,CRP,CurrentBiologics,CurrentIntralesionalSteroids,CurrentMTX,...,AgeAtVisit,prednisone,leflunomide,nomed,mtx,nan,topicals,hcq,categorical,Involvement
#SampleID,,,,,,,,,,,,,,,,,,,,,
530-0-twin-psa-plate308,TATGCCAGAGAT,CCGGACTACHVGGGTWTCTAAT,C4,0.5,NaN,NaN,NaN,0.0,NaN,0.0,...,NaN,0.0,0,0,0,1,0,0,NaN,axial
235-psa-plate307,TGCGCGCCTTCC,CCGGACTACHVGGGTWTCTAAT,E3,3,mild,NaN,NaN,0.0,0.0,0.0,...,NaN,0.0,0,0,0,1,0,0,NaN,peripheral
240-psa-plate307,GCGCACACCTTC,CCGGACTACHVGGGTWTCTAAT,F3,13,severe,NaN,NaN,0.0,0.0,NaN,...,NaN,0.0,0,0,0,1,0,0,NaN,axial
272-psa-plate307,CACGAAAGCAGG,CCGGACTACHVGGGTWTCTAAT,G3,NaN,NaN,NaN,0.3,0.0,0.0,0.0,...,NaN,0.0,0,0,0,1,0,0,NaN,axial
288-psa-plate307,ATTTGGCTCTTA,CCGGACTACHVGGGTWTCTAAT,A4,7,moderate,NaN,NaN,0.0,0.0,NaN,...,NaN,0.0,0,0,0,1,0,0,NaN,axial


In [12]:
# sample breakdown
df_merge['Involvement'].value_counts()

Involvement
peripheral    19
axial          8
Name: count, dtype: int64

In [8]:
# unnormalized OTU table level-6 also has metadata in it! nice 
# from taxa_bar_plot.qzv
#df_otu = pd.read_csv(path + 'inputs/Qiime2_0_KB_noctrl_noeiser_nocd/level-5.csv', index_col=0)
# df_otu = pd.read_csv(path + 'inputs/Q2_MSQ138_141_noctrl_noeiser_nocd_correct/level-7.csv', index_col=0)
df_otu = pd.read_csv(path + 'inputs/Q2_MSQ138_141_noctrl_noeiser_nocd_correct_pcopri/level-7.csv', index_col=0)


# determine columns to drop; i.e. keep taxa only
dropcol = []
for c in list(df_otu.columns.values):
    if c[0:3] != 'd__': # don't use k__ anymore
        dropcol.append(c)
        
# keep Diagnosis for later
keepcol = df_otu['Diagnosis']

df_otu = df_otu.drop(dropcol, axis=1)
# df_otu.to_csv(path + 'inputs/Qiime2_0_KB_batch_correct_nocd/counts_L6.csv')

# normalize the cols
df_otu = df_otu.div(df_otu.sum(axis=1), axis=0)
# df_otu.to_csv(path + 'inputs/Qiime2_0_KB_batch_correct_nocd/otu_table_L6.csv')

# reappend dx
df_otu = pd.concat([df_otu, keepcol],axis=1)
df_otu.head()

,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Christensenellales;f__CAG-74;g__;s__,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__UBA11471;g__UBA11471;s__UBA11471 sp000434215,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__Prevotella stercorea,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Christensenellales;f__Christensenellaceae;g__Christensenella;__,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Lachnospirales;f__Lachnospiraceae;g__Eubacterium_I;__,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Lachnospirales;f__Lachnospiraceae;g__Coprococcus_A_187866;s__,d__Bacteria;p__Firmicutes_D;c__Bacilli;o__Erysipelotrichales;f__Coprobacillaceae;g__Catenibacterium;s__Catenibacterium sp000437715,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Lachnospirales;f__Lachnospiraceae;g__Blautia_A_141781;__,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Oscillospirales;f__Oscillospiraceae_88309;__;__,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Muribaculaceae;g__Limisoma;s__Limisoma sp900548875,...,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Lachnospirales;f__Lachnospiraceae;g__Oribacterium;s__Oribacterium sp000160135,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Oscillospirales;f__Acutalibacteraceae;g__Ruminococcus_E;s__Ruminococcus_E sp900100595,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Peptostreptococcales;f__Anaerovoracaceae;g__Baileyella;s__,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Peptostreptococcales;f__;g__;s__,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Tissierellales;f__Peptoniphilaceae;g__Peptoniphilus_C;__,d__Bacteria;p__Firmicutes_D;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus;s__Streptococcus anginosus,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Oscillospirales;f__Ruminococcaceae;g__QAMM01;s__,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Rikenellaceae;g__Alistipes_A_871404;s__Alistipes_A_871404 ihumii,d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Pseudomonadales_650611;f__Pseudomonadaceae;g__Pseudomonas_F;s__Pseudomonas_F furukawaii,Diagnosis
index,,,,,,,,,,,,,,,,,,,,,
131-slesjo-plate308,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.026499,0.002920,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,ss
209-pso-plate307,0.000000,0.0,0.0,0.0,0.003604,0.0,0.0,0.077978,0.000484,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002725,0.0,pso
235-psa-plate307,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.227506,0.001178,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,psa
240-psa-plate307,0.000000,0.0,0.0,0.0,0.001677,0.0,0.0,0.047405,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,psa
241-pso-plate307,0.003153,0.0,0.0,0.0,0.000000,0.0,0.0,0.029957,0.024202,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,pso


In [41]:
# query lefse results
queries = ['d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Phocaeicola_A_858004;s__Phocaeicola_A_858004 dorei',
           'd__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Lachnospirales;f__Lachnospiraceae;g__Ruminococcus_B;s__Mediterraneibacter gnavus',
           'd__Bacteria;p__Verrucomicrobiota;c__Verrucomicrobiae;o__Verrucomicrobiales;f__Akkermansiaceae;g__Akkermansia;s__Akkermansia muciniphila_D_776786',
           'd__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Rikenellaceae;g__Alistipes_A_871400;s__Alistipes_A_871400 shahii',
           #pseudo = '' # not in this dataset?
           # uncl_cl = '' # lots in the lefse
           # 'd__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Clostridiales;f__Clostridiaceae_222000;__;__',
           # rumino AF48 lots of species but not AF48 or 10NS
           'd__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Lachnospirales;f__Lachnospiraceae;g__Eubacterium_G;s__Eubacterium_G ventriosum',
           # hansenii; lots of blautia but no hansenii
           # lachnospira species; two of them but not NSJ43
           # two Romboutsia but not the same
           'd__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Bacteroides_H;s__Bacteroides_H nordii',
           'd__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__Prevotella copri']


for x in df_otu.columns:
    if 'f__Clostr' in x:
        print(x)
    

d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Clostridiales;f__Clostridiaceae_222000;g__Clostridium_T;s__Clostridium_T isatidis
d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Clostridiales;f__Clostridiaceae_222000;g__Clostridium_T;__
d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Clostridiales;f__Clostridiaceae_222000;g__Clostridium_T;s__Clostridium_T disporicum_203974
d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Clostridiales;f__Clostridiaceae_222000;g__Clostridium_T;s__Clostridium_T paraputrificum_208099
d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Clostridiales;f__Clostridiaceae_222000;__;__
d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Clostridiales;f__Clostridiaceae_222000;g__Clostridium_P;s__Clostridium_P perfringens
d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Clostridiales;f__Clostridiaceae_222000;g__Clostridium_T;s__Clostridium_T chartatabidum


In [42]:
# taxa of interest
for q in queries:
    print(q)
    taxa = q.split('s__')[-1]
    
    # psa only
    df_otu_spec = pd.concat([df_otu,df_merge['Involvement']],axis=1)
    df_otu_spec = df_otu_spec[df_otu_spec['Involvement'].isin(['axial','peripheral'])]
    
    # KW test
    print(scipy.stats.kruskal(*list(df_otu_spec.groupby('Involvement')[q].apply(list).values), nan_policy='propagate', axis=0, keepdims=False))
    
    # do sns 
    plt.figure(figsize=(3, 2)) 
    ax = sns.boxplot(data=df_otu_spec, x='Involvement', y=q)
    sns.swarmplot(data=df_otu_spec, x='Involvement', y=q, palette='dark:grey', hue=None)
    ax.set_ylabel("Abundance",fontsize=8)
    ax.set_xlabel("Disease",fontsize=8)
    ax.tick_params(labelsize=6)
    plt.title(taxa,fontsize=8)
    ax.figure.savefig(path + 'outputs/jobs00/plots/' + taxa + '.pdf')
    sns.despine()
    plt.tight_layout()
    plt.close()


d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Phocaeicola_A_858004;s__Phocaeicola_A_858004 dorei
KruskalResult(statistic=1.7221231043711078, pvalue=0.18941998406826477)
d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Lachnospirales;f__Lachnospiraceae;g__Ruminococcus_B;s__Mediterraneibacter gnavus
KruskalResult(statistic=4.932692307692287, pvalue=0.02635322873201666)
d__Bacteria;p__Verrucomicrobiota;c__Verrucomicrobiae;o__Verrucomicrobiales;f__Akkermansiaceae;g__Akkermansia;s__Akkermansia muciniphila_D_776786
KruskalResult(statistic=0.23656924569487242, pvalue=0.626695041654405)
d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Rikenellaceae;g__Alistipes_A_871400;s__Alistipes_A_871400 shahii
KruskalResult(statistic=0.19618780370954456, pvalue=0.6578156986845634)
d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Lachnospirales;f__Lachnospiraceae;g__Eubacterium_G;s__Eubacterium_G ventriosum
KruskalResult(statistic=0.5116116905166119, p

In [75]:
# stercoria, other taxa of interest
# take taxonomy.qza to map asv to taxonomic classification
# rep_seqs.qza contains mapping of sequnces to asv
# we want to take the sequences mapped to g prevotella and see where it blasts in ncbi

# obtain key (asv) mapping to read (sequence)
fasta = path + 'inputs/Q2_MSQ138_141_noctrl_noeiser_nocd_correct/bbe41c4b-561b-4b6d-8d4a-d1463dc2639e/data/dna-sequences.fasta'
from Bio import SeqIO

def parse_fasta(filename):
    sequences = {}
    for record in SeqIO.parse(filename, "fasta"):
        sequences[record.id] = str(record.seq)
    return sequences

# usage
sequences = parse_fasta(fasta)

#print(len(sequences)) # 6521 sequences mapped to asvs
#first_entry = next(iter(sequences.items()))
#print(first_entry)

# obtain mapping of taxa to asv
taxonomy = path + 'inputs/Q2_MSQ138_141_noctrl_noeiser_nocd_correct/118f854f-e182-4d1e-b4aa-61ceed48b642/data/taxonomy.tsv'

df_taxonomy = pd.read_csv(taxonomy, sep='\t')
# Feature ID, Taxon, Confidence

prev = []
pcopri = []
for t in df_taxonomy.Taxon:
    if 'Prevotella' in t:
        prev.append(t)
    if 'copri' in t:
        pcopri.append(t)
        
print(len(prev)) # 249 sequences map to prevotella et al.
print(len(pcopri)) # 65 of those map to copri
query = ['d__Bacteria; p__Bacteroidota; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Prevotella; s__',
         'd__Bacteria; p__Bacteroidota; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Prevotella',
         'd__Bacteria; p__Bacteroidota; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Prevotella; __']

# find asvs to keep
def filter_fasta_by_names(input_filename, output_filename, names_to_keep):
    with open(output_filename, 'w') as output_file:
        for record in SeqIO.parse(input_filename, 'fasta'):
            if record.id in names_to_keep:
                SeqIO.write(record, output_file, 'fasta')

# Example usage:
input_filename = fasta
output_filename = path + 'inputs/Q2_MSQ138_141_noctrl_noeiser_nocd_correct/prevotella_filt.fasta'

# subset on prevotella
df = df[df['Taxon'].isin(query)]
names_to_keep = list(df['Feature ID'].values)
print(len(names_to_keep))
filter_fasta_by_names(input_filename, output_filename, names_to_keep)

df.head()
#for p in prev:
#    print(p.split('g__Prevotella')[-1])
# 
# print(prev)





249
65
62


,Feature ID,Taxon,Confidence
117,059c2b3a539b37a36dc4807edd8d4f40,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.929739
118,059fb473103d183bfb0417da22a25447,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.978016
316,0cb071d8b433f21cb1d13bc58c9f0e12,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.997966
343,0da398195a3b83569b61540a2f262f80,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.999984
381,0efa60ab73b0d539f66e195f803e09a7,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.968758


In [76]:
# stercoria, other taxa of interest
# take taxonomy.qza to map asv to taxonomic classification
# rep_seqs.qza contains mapping of sequnces to asv
# we want to take the sequences mapped to g prevotella and see where it blasts in ncbi

# obtain key (asv) mapping to read (sequence)
fasta = path + 'inputs/Q2_MSQ138_141_noctrl_noeiser_nocd_correct/bbe41c4b-561b-4b6d-8d4a-d1463dc2639e/data/dna-sequences.fasta'
from Bio import SeqIO

def parse_fasta(filename):
    sequences = {}
    for record in SeqIO.parse(filename, "fasta"):
        sequences[record.id] = str(record.seq)
    return sequences

# usage
sequences = parse_fasta(fasta)

#print(len(sequences)) # 6521 sequences mapped to asvs
#first_entry = next(iter(sequences.items()))
#print(first_entry)

# obtain mapping of taxa to asv
taxonomy = path + 'inputs/Q2_MSQ138_141_noctrl_noeiser_nocd_correct/118f854f-e182-4d1e-b4aa-61ceed48b642/data/taxonomy.tsv'

df_taxonomy = pd.read_csv(taxonomy, sep='\t')
# Feature ID, Taxon, Confidence

prev = []
pcopri = []
for t in df_taxonomy.Taxon:
    if 'Prevotella' in t:
        prev.append(t)
    if 'copri' in t:
        pcopri.append(t)
        
print(len(prev)) # 249 sequences map to prevotella et al.
print(len(pcopri)) # 65 of those map to copri
query = ['d__Bacteria; p__Bacteroidota; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Prevotella; s__',
         'd__Bacteria; p__Bacteroidota; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Prevotella']

# find asvs to keep
def filter_fasta_by_names(input_filename, output_filename, names_to_keep):
    with open(output_filename, 'w') as output_file:
        for record in SeqIO.parse(input_filename, 'fasta'):
            if record.id in names_to_keep:
                SeqIO.write(record, output_file, 'fasta')

# Example usage:
input_filename = fasta
output_filename = path + 'inputs/Q2_MSQ138_141_noctrl_noeiser_nocd_correct/prevotella_filt.fasta'

# subset on prevotella
df = df[df['Taxon'].isin(query)]
names_to_keep = list(df['Feature ID'].values)
print(len(names_to_keep)) # 62 sequences asvs that map to uncl Prevotella species to check  
filter_fasta_by_names(input_filename, output_filename, names_to_keep)

# these are the seqs that will be blasted!
df.head()

249
65
62


,Feature ID,Taxon,Confidence
117,059c2b3a539b37a36dc4807edd8d4f40,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.929739
118,059fb473103d183bfb0417da22a25447,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.978016
316,0cb071d8b433f21cb1d13bc58c9f0e12,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.997966
343,0da398195a3b83569b61540a2f262f80,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.999984
381,0efa60ab73b0d539f66e195f803e09a7,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.968758


In [58]:
# analyze blast results
from Bio.Blast import NCBIXML

def parse_blast_alignment(blast_output_file):
    alignments = []
    with open(blast_output_file, 'r') as result_handle:
        blast_records = NCBIXML.parse(result_handle)
        for blast_record in blast_records:
            for alignment in blast_record.alignments:
                for hsp in alignment.hsps:
                    alignments.append({
                        'query_id': blast_record.query,
                        'subject_id': alignment.title,
                        'e_value': hsp.expect,
                        'alignment_length': hsp.align_length,
                        # 'alignment': hsp.align,
                        # Add more attributes as needed
                    })
    return alignments

# get blasted files
blast_output_file =  path + 'inputs/Q2_MSQ138_141_noctrl_noeiser_nocd_correct/5M0KTYMR013-Alignment.xml'
parsed_alignments = parse_blast_alignment(blast_output_file)

copri = []
sterc = []
for a in parsed_alignments:
    if 'copri' in a['subject_id']:
        copri.append(a['query_id'])
    if 'sterc' in a['subject_id']:
        sterc.append(a['subject_id'])

# 62 blasted seqs, 37 mapped to copri and 0 to stercoria
print(len(copri))
print(len(sterc))

37
0


In [68]:
# make a new taxonomy table that maps the asvs that were uncl that mapped to copri via blast
df_new = df_taxonomy.copy()
pcopri = 'd__Bacteria; p__Bacteroidota; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Prevotella; s__Prevotella copri'
df_new.loc[df_new['Feature ID'].isin(copri), 'Taxon'] = pcopri
df_new.to_csv(path + 'inputs/gg2_pcopri_blast_taxonomy.tsv', index = False, sep='\t')
df_new.head()
# copri

,Feature ID,Taxon,Confidence
0,000db46fcff4701589d112bc2a6044de,d__Bacteria; p__Firmicutes_A; c__Clostridia_25...,0.999982
1,00127d77ae557f7da2ce9f44c0ef5a61,d__Bacteria; p__Verrucomicrobiota; c__Verrucom...,0.881100
2,001343881df5dc325f1eb57a0c3c9ec9,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.926825
3,001bb5d73b004c8f3292e4c9f147cf8f,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.999271
4,002828baa9cba6b0986decec29cc6381,d__Bacteria; p__Firmicutes_A; c__Clostridia_25...,0.998879


In [46]:
# split df otu into metadata nad otu vars
# quant vars
vars = ['BSA','CRP','ESR','SJC','TJC','DAS28','Involvement']

df_otu = pd.read_csv(path + 'inputs/Q2_MSQ138_141_psa/level-6.csv', index_col=0)

# determine columns to drop; i.e. keep taxa only
dropcol = []
for c in list(df_otu.columns.values):
    if c[0:3] != 'd__': # don't use k__ anymore
        dropcol.append(c)

# partition df_meta
df_meta = df_otu[vars]

# drop non otu
df_otu = df_otu.drop(dropcol, axis=1)

# normalize the cols
df_otu = df_otu.div(df_otu.sum(axis=1), axis=0)
# df_otu.to_csv(path + 'inputs/Qiime2_0_KB_batch_correct_nocd/otu_table_L6.csv')

# reappend dx
df_otu = pd.concat([df_otu, keepcol],axis=1)
df_otu.head()

,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Christensenellales;f__Christensenellaceae;g__Christensenella,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Lachnospirales;f__Lachnospiraceae;g__Blautia_A_141781,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Oscillospirales;f__Ruminococcaceae;g__Bittarella,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Christensenellales;f__CAG-74;g__,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Lachnospirales;f__Lachnospiraceae;g__,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Oscillospirales;f__Acutalibacteraceae;g__Caproiciproducens,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Lachnospirales;f__Lachnospiraceae;g__Blautia_A_141780,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Muribaculaceae;g__Paramuribaculum,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Oscillospirales;f__CAG-272;g__RUG13077,...,d__Bacteria;p__Actinobacteriota;c__Actinomycetia;o__Mycobacteriales;f__Mycobacteriaceae;g__Corynebacterium,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__UBA6382,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Oscillospirales;f__Ruminococcaceae;g__Pygmaiobacter,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Lachnospirales;f__Lachnospiraceae;g__CAG-45,d__Bacteria;p__Firmicutes_D;c__Bacilli;o__RF39;f__UBA660;g__CAG-417,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Tissierellales;f__Peptoniphilaceae;g__Peptoniphilus_C,d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Burkholderiales_592524;__;__,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__UBA6398,d__Bacteria;p__Firmicutes_A;c__Clostridia_258483;o__Oscillospirales;f__Butyricicoccaceae;g__Butyricicoccus_A_77030,Involvement
index,,,,,,,,,,,,,,,,,,,,,
235-psa-plate307,0.000168,0.0,0.268243,0.0,0.000000,0.000000,0.0,0.003114,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,peripheral
240-psa-plate307,0.000000,0.0,0.062114,0.0,0.000000,0.000686,0.0,0.000000,0.005183,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000533,axial
272-psa-plate307,0.002815,0.0,0.242649,0.0,0.000000,0.000000,0.0,0.001323,0.002001,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,axial
288-psa-plate307,0.000000,0.0,0.298877,0.0,0.000339,0.000000,0.0,0.002160,0.001975,0.0,...,0.000123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,axial
290-psa-plate307,0.000000,0.0,0.029768,0.0,0.000000,0.000000,0.0,0.008229,0.031284,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,peripheral


,BarcodeSequence,LinkerPrimerSequence,AmpliconWell,BSA,BSASeverityByBSA,CCPtiter,CRP,CurrentBiologics,CurrentIntralesionalSteroids,CurrentMTX,...,AgeAtVisit,prednisone,leflunomide,nomed,mtx,nan,topicals,hcq,categorical,Involvement
#SampleID,,,,,,,,,,,,,,,,,,,,,
530-0-twin-psa-plate308,TATGCCAGAGAT,CCGGACTACHVGGGTWTCTAAT,C4,0.5,NaN,NaN,NaN,0.0,NaN,0.0,...,NaN,0.0,0,0,0,1,0,0,NaN,axial
235-psa-plate307,TGCGCGCCTTCC,CCGGACTACHVGGGTWTCTAAT,E3,3,mild,NaN,NaN,0.0,0.0,0.0,...,NaN,0.0,0,0,0,1,0,0,NaN,peripheral
240-psa-plate307,GCGCACACCTTC,CCGGACTACHVGGGTWTCTAAT,F3,13,severe,NaN,NaN,0.0,0.0,NaN,...,NaN,0.0,0,0,0,1,0,0,NaN,axial
272-psa-plate307,CACGAAAGCAGG,CCGGACTACHVGGGTWTCTAAT,G3,NaN,NaN,NaN,0.3,0.0,0.0,0.0,...,NaN,0.0,0,0,0,1,0,0,NaN,axial
288-psa-plate307,ATTTGGCTCTTA,CCGGACTACHVGGGTWTCTAAT,A4,7,moderate,NaN,NaN,0.0,0.0,NaN,...,NaN,0.0,0,0,0,1,0,0,NaN,axial
290-psa-plate307,CGCGTCAAACTA,CCGGACTACHVGGGTWTCTAAT,B4,3,moderate,NaN,1.1,0.0,0.0,0.0,...,NaN,0.0,0,0,0,1,0,0,NaN,peripheral
305-psa-plate307,ATGTTCCTCATC,CCGGACTACHVGGGTWTCTAAT,C4,50,severe,NaN,NaN,0.0,0.0,NaN,...,NaN,0.0,0,0,0,1,0,0,NaN,peripheral
306-psa-plate307,TCCGTGGTATAG,CCGGACTACHVGGGTWTCTAAT,D4,1.5,mild,NaN,NaN,0.0,0.0,0.0,...,NaN,0.0,0,0,0,1,0,0,NaN,peripheral
308-psa-plate307,GTTGGTTGGCAT,CCGGACTACHVGGGTWTCTAAT,E4,4.5,moderate,NaN,28,0.0,0.0,0.0,...,NaN,0.0,0,0,0,1,0,0,NaN,peripheral
